In [ ]:
import os
import lephare as lp
import numpy as np
import matplotlib

%matplotlib inline
from matplotlib import pylab as plt

# Empirical Model

TODO

# Physical Model

The physical model relies on definite, even if approximate, prescriptions to evaluate the contributions of nebular emission and emission lines. It needs an SED that has coverage below ~1200. As a result, we will start by loading the Bruzual&Charlot library distributed in [lephare-data](https://github.com/lephare-photoz/lephare-data)

In [ ]:
config = lp.default_cosmos_config.copy()
lp.data_retrieval.get_auxiliary_data(
    keymap=config, additional_files=["sed/GAL/BC03_CHAB/bc2003_lr_m42_chab_tau01_dust00.ised_ASCII"]
)

In [ ]:
seds = lp.readBC03(
    os.path.join(os.environ["LEPHAREDIR"], "sed/GAL/BC03_CHAB/bc2003_lr_m42_chab_tau01_dust00.ised_ASCII"),
    0,  # id of the template
    [-999, -999],  # dummy values if no age range is requested
)

In [ ]:
fig, ax = plt.subplots()
[ax.loglog(*sed.data()) for sed in seds]
plt.xlabel(r"$\lambda$ [A]")
plt.ylabel(r"[ergs s$^{-1}$ Hz$^{-1}$]");

## Computing the number of ionizing photons

We first need to compute the number flux of photons able to ionize the hyddrogen atom. The code `SED::calc_ph` actually stores the number flux for four ionization edges: HeII, HeI, H, and H2 first excitation state. For a given SED, this amounts to compute the integral
$\int_0^{w_i} SED(\lambda)\cdot \frac{\lambda}{hc}\,d\lambda,$
where $w_i=\,227.8,\, 505.6,\, 911.6$, and $1108.7$ A for HeII, HeI, H, and H2 respectively, and where $hc$ is in ergs.A. 
This normalization assumes that the SED is only self consistent for sed provided in ergs/cm2/s/A, up to an overall scaling factor obtained during a fit to a set of photometric measurements. 
Results are stored in the $q_i$ array member of size 4 of the SED instance.


In [ ]:
sed = seds[20]  # we take any sed but seds[0] which is empty
print(f"SED age: {sed.age}")
sed.calc_ph()
print(f"Number flux of ionizing photons for HeII, HeI, H, and H2: {sed.qi}")

## Estimating the nebular continuum contribution

Stellar population synthesis code usually do not account for nebular emission in synthesizing a galaxy spectrum. 
Nebular emission comes from HII regions ionized by very energetic photons ($h\nu\geq 13.6$ eV), typically coming from young blue stars.
For typical HII regions (temperature in the range 5000 to 20000 K and density below $10^4$cm$^{-3}$), collisional deexcitations are negligible compared to radiative deexcitation. Recombination then yields a continum and line emission that needs to be taken into account in a photometric fit, as determined e.g. by [Shaerer and de Barros 2009](https://ui.adsabs.harvard.edu/abs/2009A%26A...502..423S/abstract).

Of course a software like `LePHARE` cannot implement a full treatment of recombination processes. It thus follows the treatment of [Shearer and Vacca 1998](https://ui.adsabs.harvard.edu/abs/1998ApJ...497..618S/abstract). We consider only "case B" nebular emission, which means that the nebula is opaque to recombination photons (and thus we are in a ionization-recombination equilibrium), with an electron density and temperature set to $n_e=100$cm$^{-3}$ and $T_e=10000\,$K
the monochromatic luminosity can be written
$$
L_\gamma = \frac{c}{\lambda^2}\frac{\gamma_{total}}{\alpha_B}f_\gamma Q_0
$$
where:
- $\gamma_{total}$ is the total nebular continuum emission coefficient, that we discuss next.
- $\alpha_B$ is the "case B" recombination coefficient for Hydrogen. Its value, $2.59\, 10^{-13}$ is taken from table 2.1 of the [Osterbrock and Ferland 2006 mononograph](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://dokumen.pub/astrophysics-of-gaseous-nebulae-and-active-galactic-nuclei-2nbsped-1891389343-9781891389344-z-6872184.html&ved=2ahUKEwjjz7eogpuRAxWdK_sDHUCDKy8QFnoECBgQAQ&usg=AOvVaw3TPx598-tx0Mxi9-dDdHK0), and is valid for $T_e=10000$K.
- $Q_0$ is the number of ionizing photons (the Lyman continuum) computed above, and $f_{\gamma}$ is the fraction of these photons absorbed by the gas; it is set to 1. in the code.

For a He abundance of 10% by number relative to hydrogen, and neglecting HeIII contribution, $\gamma_{total}$ reads:
$$
\gamma_{total} = \gamma_\nu(HI) + 0.1\,\gamma_\nu(HeI) + \gamma_\nu(2q)\quad ,
$$
where the last contribution comes from the 2 photon deexcitation of H 2 2s channel. The values of the three $\gamma$ coefficient are tabulated in `emission_lines.h` and are assembled as follows:
- [Aller 1984](https://link.springer.com/book/10.1007/978-94-010-9639-3) provides in table 4.9 values for the coefficients at $T_e=10000_,$K from $\lambda=1000\,$A to $\lambda=10000.1\,$A, in units of $10^{-40}\,$ergs cm$^3$ s$^{-1}$ Hz$^{-1}$.
- [Ferland 1980](https://articles.adsabs.harvard.edu/pdf/1980PASP...92..596F) supplement these with values for HI between 14591 and 132173 A.
- We assume $\gamma_\nu(HeI)= \gamma_\nu(HI)$  and $\gamma_\nu(2q) = 0$ above 10000.1 A.

We can access and plot the resulting curves:

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(lp.ga_lamb, lp.ga_H_val, label=r"$\gamma_\nu(HI)$")
ax.plot(lp.ga_lamb, lp.ga_2q_val, label=r"$\gamma_\nu(2q)$")
ax.plot(lp.ga_lamb, np.array(lp.ga_HeI_val), label=r"$\gamma_\nu(HeI)$")
ax.plot(
    lp.ga_lamb,
    np.array(lp.ga_H_val) + np.array(lp.ga_2q_val) + 0.1 * np.array(lp.ga_HeI_val),
    label=r"$\gamma_{tot}$",
)
plt.ylabel("emission coefficient [$10^{-40}$erg cm$^3$ s$^{-1}$ Hz$^{-1}$]")
plt.xlabel(r"$\lambda$ [A]")
plt.legend();

The resulting effect on an SED can now be illustrated

In [ ]:
print(f"galaxy age: {sed.age} yr")
x, y = sed.data()
cont = sed.add_neb_cont()
x2, y2 = sed.data()
fig, ax = plt.subplots()
ax.semilogy(x, y, label="stellar continuum")
ax.semilogy(x2, y2, label="total continuum")
ax.semilogy(x2, cont, label="nebular continuum")
plt.xlim(0, 10000)
plt.ylim(1.0e-12, 1.0e-8)
plt.xlabel(r"$\lambda$ [A]")
plt.ylabel(r"[ergs $s^{-1}$ Hz$^{-1}$]")
plt.legend();

## Computing the emission line contribution 

Emission line recombination luminosities are compiled from two references :
- The hydrogen Balmer Paschen and Bracket recombination lines can be found on table 4.2 of [Osterbrock and Ferland 2006 mononograph](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://dokumen.pub/astrophysics-of-gaseous-nebulae-and-active-galactic-nuclei-2nbsped-1891389343-9781891389344-z-6872184.html&ved=2ahUKEwjjz7eogpuRAxWdK_sDHUCDKy8QFnoECBgQAQ&usg=AOvVaw3TPx598-tx0Mxi9-dDdHK0)
- Non hydrogen emission lines come from [Anders and Fritze-v. Alvensleben 2003](https://ui.adsabs.harvard.edu/abs/2003A%26A...401.1063A/abstract)
In both cases, the luminosities are provided in units of the H$\beta$ luminosity, which can be computed as

$$
L_{H_\beta} = \frac{hc}{\lambda_{H_\beta}}\frac{\alpha^{eff}_{H_\beta}}{\alpha_B}f_\gamma Q_0
$$
where $\alpha^{eff}_{H_\beta}=3.03\,10^{-14}$ cm$^3$s$^{-1}$ for $T_e=10000\,$K and case B (table 4.2 in Osterbrock and Ferland again).

We can visualize the lines in the following way

In [ ]:
sed.generateEmPhys(sed.zmet, sed.qi[2])
x = [onel.lamb for onel in sed.fac_line]
y = [onel.val for onel in sed.fac_line]
fig, ax = plt.subplots()
for xx, yy in zip(x, y):
    ax.plot([xx, xx], [0, yy], "-")

These are then transformed into gaussians using an arbitrary 100km/s line dispersion, 40 steps and a span of 6 times the line dispersion.

In [ ]:
linesed = lp.GalSED(sed)
linesed.generateEmSpectra(40)
fig, ax = plt.subplots()
ax.plot(*linesed.data())
plt.xlim(0, 40000)
# plt.ylim(0, 1.e-7)